# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# SearchHit + Google Custom Search

In [ ]:
from dotenv import load_dotenv
import os

from intergrax.websearch.schemas.query_spec import QuerySpec
from intergrax.websearch.providers.google_cse_provider import GoogleCSEProvider

load_dotenv()

os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY")
os.environ["GOOGLE_CSE_CX"] = os.getenv("GOOGLE_CSE_CX")

spec = QuerySpec(
    query="What is LangGraph and how does it differ from LangChain?",
    top_k=8,
    locale="pl-PL",
    region="pl-PL",
    language="pl",
    safe_search=True,
)


provider = GoogleCSEProvider()
hits = provider.search(spec)

print(f"Found {len(hits)} results\n")

for i, hit in enumerate(hits, start=1):
    print(f"Result {i}:")
    print(f"  provider   : {hit.provider}")
    print(f"  rank       : {hit.rank}")
    print(f"  title      : {hit.title}")
    print(f"  url        : {hit.url}")
    print(f"  snippet    : {hit.snippet}")
    print(f"  domain     : {hit.domain()}")
    print(f"  published  : {hit.published_at}")
    print("-" * 80)


# WebSearchExecutor with Google and Bing Search

In [10]:
from dotenv import load_dotenv
import os

from intergrax.websearch.service.websearch_executor import WebSearchExecutor

# 1. Load environment variables
load_dotenv()

os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY")
os.environ["GOOGLE_CSE_CX"] = os.getenv("GOOGLE_CSE_CX")
os.environ["BING_SEARCH_V7_API_KEY"] = os.getenv("BING_SEARCH_V7_API_KEY") or ""

# 2. Create configurable executor
#    Here you can decide which providers to use and set defaults.
executor = WebSearchExecutor(
    enable_google_cse=True,
    enable_bing_web=True,
    default_top_k=8,
    default_locale="pl-PL",
    default_region="pl-PL",
    default_language="pl",
    default_safe_search=True,
    max_text_chars=4000,
)

# 3. Define user query
query = "What is LangGraph and how does it differ from LangChain?"

# 4. Run async search
#    You can override defaults here (top_k, language, etc.).
web_docs = await executor.search_async(
    query=query,
    top_k=8,          # override default_top_k if needed
    language="en",    # override default_language if needed
    top_n_fetch=None, # None -> use top_k
    serialize=True,   # return list[dict] ready for printing/LLM
)

# 5. Inspect results
for i, doc in enumerate(web_docs, start=1):
    print(f"=== DOCUMENT {i} ===")
    for key, value in doc.items():
        print(f"{key}: {value}")
    print("\n\n")


=== DOCUMENT 1 ===
provider: google_cse
rank: 2
source_rank: 2
quality_score: 2.0
title: ⚙️LangChain vs. LangGraph: A Comparative Analysis | by Tahir | Medium
url: https://medium.com/@tahirbalarabe2/%EF%B8%8Flangchain-vs-langgraph-a-comparative-analysis-ce7749a80d9c
snippet: 13 lut 2025 ... LangGraph uses a graph structure that allows for loops and revisiting previous states, making it better for interactive systems. Components
description: ⚙️LangChain vs. LangGraph: A Comparative Analysis Full Credit to IBM Technology If you’re building applications with large language models, you’ll likely come across two frameworks: LangChain …
lang: en
domain: medium.com
published_at: 2025-10-08T10:14:15.807000+00:00
fetched_at: 2025-11-14T06:55:10.245967
text: ⚙️LangChain vs. LangGraph: A Comparative Analysis | by Tahir | Medium
Sitemap
Open in app
Sign up
Sign in
Medium Logo
Write
Search
Sign up
Sign in
⚙️LangChain vs. LangGraph: A Comparative Analysis
Tahir
7 min read
·
Feb 14, 2025
--
5
Listen


# LangGraph using WebSearchExecutor

In [2]:
from dotenv import load_dotenv
import os
from intergrax.websearch.integration.langgraph_nodes import (
    WebSearchNode,
    WebSearchState,
)

# 1. Load environment
load_dotenv()

os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY")
os.environ["GOOGLE_CSE_CX"] = os.getenv("GOOGLE_CSE_CX")
os.environ["BING_SEARCH_V7_API_KEY"] = os.getenv("BING_SEARCH_V7_API_KEY") or ""

# 2. Configure node (OPTIONAL custom settings)
node = WebSearchNode(
    enable_google_cse=True,
    enable_bing_web=True,
    default_top_k=5,
    default_language="en",
    default_locale="en-US",
    default_region="US",
    max_text_chars=2000,
)

# 3. LangGraph-compatible state
state: WebSearchState = {
    "messages": [],
    "user_question": "What is LangGraph and how does it differ from LangChain?",
}

# 4. Execute
new_state = await node.run_async(state)

web_docs = new_state.get("websearch_docs", [])

print(f"Found: {len(web_docs)} results\n")

for i, doc in enumerate(web_docs, start=1):
    print(f"===== RESULT {i} =====")
    for k, v in doc.items():
        print(f"{k}: {v}")
    print("\n")


Found: 5 results

===== RESULT 1 =====
provider: google_cse
rank: 2
source_rank: 2
quality_score: 2.0
title: ⚙️LangChain vs. LangGraph: A Comparative Analysis | by Tahir | Medium
url: https://medium.com/@tahirbalarabe2/%EF%B8%8Flangchain-vs-langgraph-a-comparative-analysis-ce7749a80d9c
snippet: Feb 13, 2025 ... LangGraph uses a graph structure that allows for loops and revisiting previous states, making it better for interactive systems. Components
description: ⚙️LangChain vs. LangGraph: A Comparative Analysis Full Credit to IBM Technology If you’re building applications with large language models, you’ll likely come across two frameworks: LangChain …
lang: en
domain: medium.com
published_at: 2025-10-08T10:14:15.807000+00:00
fetched_at: 2025-11-14T08:03:56.914302
text: ⚙️LangChain vs. LangGraph: A Comparative Analysis | by Tahir | Medium
Sitemap
Open in app
Sign up
Sign in
Medium Logo
Write
Search
Sign up
Sign in
⚙️LangChain vs. LangGraph: A Comparative Analysis
Tahir
7 min read
·
Feb 

# Execute full Intergrax web-augmented reasoning pipeline: retrieval, context building, and LLM response generation.

In [2]:
from dotenv import load_dotenv
import os

from langchain_ollama import ChatOllama

from intergrax.llm_adapters import LangChainOllamaAdapter
from intergrax.websearch.service.websearch_executor import WebSearchExecutor
from intergrax.websearch.context.websearch_context_builder import WebSearchContextBuilder
from intergrax.websearch.service.websearch_answerer import WebSearchAnswerer

# 1. Load environment variables
load_dotenv()

os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY")
os.environ["GOOGLE_CSE_CX"] = os.getenv("GOOGLE_CSE_CX")
os.environ["BING_SEARCH_V7_API_KEY"] = os.getenv("BING_SEARCH_V7_API_KEY") or ""
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 2. Create OpenAI client and adapter via registry
adapter = LangChainOllamaAdapter(
    chat=ChatOllama(
        model="llama3.1:latest"
    ),
)

# 3. Configure web search executor
executor = WebSearchExecutor(
    enable_google_cse=True,
    enable_bing_web=True,
    default_top_k=6,
    default_locale="pl-PL",
    default_region="pl-PL",
    default_language="pl",
    default_safe_search=True,
    max_text_chars=2000,
)

# 4. Optional: customize how context is built
context_builder = WebSearchContextBuilder(
    max_docs=4,
    max_chars_per_doc=1500,
    include_snippet=True,
    include_url=True,
    source_label_prefix="Source",
)

# 5. Create high-level web search answerer
answerer = WebSearchAnswerer(
    executor=executor,
    adapter=adapter,
    context_builder=context_builder,
    answer_language="pl",
)

# 6. Ask a question – FULL FLOW: search → context → LLM
question = "What is LangGraph and how does it differ from LangChain?"

result = await answerer.answer_async(
    question=question,
    top_k=6,
    system_prompt_override=(
        "You are a senior AI systems architect with deep knowledge of agent frameworks, LLM orchestration, and retrieval-augmented reasoning.\n\n"
        "Your ONLY source of truth is the provided retrieved web content. You MUST base every statement strictly and verifiably on these sources.\n"
        "Do NOT guess, hallucinate, invent facts, or use prior knowledge outside the retrieved context.\n\n"

        "Response rules:\n"
        "1. The answer MUST be detailed, analytical, and technically precise.\n"
        "2. You MUST explain concepts clearly and compare facts only when present in the sources.\n"
        "3. If information is not available in the provided sources, explicitly write: \"No available information in the retrieved sources.\"\n"
        "4. Do not repeat or rephrase without adding value.\n"
        "5. Use structured formatting: headings, sections, bullet points, comparisons, and conclusions.\n"
        "6. Reference sources inline using the format: [Source n].\n"
        "7. Minimum length requirement: The final response MUST contain **at least 1000 words**.\n"
        "8. The tone must be formal, objective, and academically rigorous.\n"
    )
)


print("=== FINAL ANSWER ===\n")
print(result["answer"])

print("\n=== SOURCES (web_docs) ===\n")
for i, doc in enumerate(result["web_docs"], start=1):
    print(f"[Source {i}] {doc.get('title')} - {doc.get('url')}")


=== FINAL ANSWER ===

**LangGraph i LangChain: Podstawowe Informacje**

LangGraph jest platformą do tworzenia agentów AI, która została opisana w [Source 2]. Jest ona częścią biblioteki LangChain, o której mowa w [Source 3] jako kursie dla programistów chcących tworzyć aplikacje GenAI od podstaw. LangGraph jest używany do tworzenia agentów AI, a jego celem jest ułatwienie tworzenia praktycznych aplikacji z wykorzystaniem modeli LLM.

**Różnice między LangGraph i LangChain**

LangGraph jest platformą specyficzną dla tworzenia agentów AI, natomiast LangChain to ogólna biblioteka do tworzenia aplikacji GenAI. LangChain zawiera wiele narzędzi i bibliotek, w tym LangGraph, który jest używany do tworzenia agentów AI. LangChain oferuje szeroki zakres funkcji, w tym biblioteki, modele i narzędzia do tworzenia aplikacji GenAI.

**Użycie LangGraph jako usługi API**

Zgodnie z [Source 4], LangGraph może być używany jako usługę API. Można go hostować za pomocą Streamlit lub FastAPI, co pozwala na 

# Cache

In [2]:
from dotenv import load_dotenv
import os

from langchain_ollama import ChatOllama

from intergrax.llm_adapters import LangChainOllamaAdapter
from intergrax.websearch.service.websearch_executor import WebSearchExecutor
from intergrax.websearch.context.websearch_context_builder import WebSearchContextBuilder
from intergrax.websearch.service.websearch_answerer import WebSearchAnswerer
from intergrax.websearch.cache import InMemoryQueryCache

load_dotenv()

os.environ["GOOGLE_CSE_API_KEY"] = os.getenv("GOOGLE_CSE_API_KEY")
os.environ["GOOGLE_CSE_CX"] = os.getenv("GOOGLE_CSE_CX")
os.environ["BING_SEARCH_V7_API_KEY"] = os.getenv("BING_SEARCH_V7_API_KEY") or ""

# 1. Create cache
query_cache = InMemoryQueryCache(
    max_entries=256,
    ttl_seconds=600,  # 10 minutes
)

# 2. LLM adapter
adapter = LangChainOllamaAdapter(
    chat=ChatOllama(model="llama3:latest"),
)

# 3. Web search executor with cache
executor = WebSearchExecutor(
    enable_google_cse=True,
    enable_bing_web=True,
    default_top_k=6,
    default_locale="pl-PL",
    default_region="pl-PL",
    default_language="pl",
    default_safe_search=True,
    max_text_chars=2000,
    query_cache=query_cache,
)

# 4. Context builder
context_builder = WebSearchContextBuilder(
    max_docs=4,
    max_chars_per_doc=1500,
    include_snippet=True,
    include_url=True,
    source_label_prefix="Source",
)

# 5. Answerer
answerer = WebSearchAnswerer(
    executor=executor,
    adapter=adapter,
    context_builder=context_builder,
    answer_language="pl",
)

question = "What is LangGraph and how does it differ from LangChain?"

result = await answerer.answer_async(
    question=question,
    top_k=6,
    temperature=0.2,
    max_tokens=6000,
)

print("=== FINAL ANSWER ===\n")
print(result["answer"])


=== FINAL ANSWER ===

LangGraph jest platformą do tworzenia agentów AI, która obsługuje różne typy interfejsów, takie jak React, SVG, PNG i WebP. LangGraph jest częścią pakietu LangChain, który towarzyszy Gemini API [Source 1]. LangGraph oferuje wiele wariantów ikon, które mogą być pobierane w różnych formatach i trybach.

LangGraph jest używany do tworzenia aplikacji GenAI od podstaw, a także do hostowania agentów jako usługę API [Source 2]. LangChain to platforma do tworzenia aplikacji GenAI, która obsługuje różne typy interfejsów, w tym LangGraph [Source 3].

Sources do not provide information on how LangGraph differs from LangChain.
